# Oppgave 1

In [195]:
from typing import Callable

hmac_key = 0b1001
hmac_ipad = 0b0011
hmac_opad = 0b0101

def ms_hash(x: int) -> int:
    return ((x*x & 0xFF) & 0b00111100) >> 2

def ms_hmac(k: int, x: int):
    if k > 0xFF:
        k = ms_hash(k)
    return ms_hash(
        (k ^ hmac_opad) << 4 |
        ms_hash(
            (k ^ hmac_ipad) << x.bit_length() |
            x
        )
    )

## a)

In [196]:
format(ms_hmac(hmac_key, 0b0110), "#06b")

'0b0100'

## b)

In [197]:
format(ms_hmac(hmac_key, 0b0111), "#06b")

'0b0100'

HMAC av denne meldingen stemmer overens med algoritmen som brukes, dermed er det mulig at meldingen er autentisk. Allikevel er det viktig å ta i betrakting at vi bruker en algoritme med blokkstørrelse lik 4 bits, noe som gjør kollisjoner meget sannsynlige.

# Oppgave 2

In [198]:
def ceasar(x: int) -> int:
    return (x + 3) & 0xF

def ceasar_cbc_mac(x: int):
    p = ceasar(x & 0xF)
    x >>= 4
    while x > 0:
        p = ceasar((x & 0xF) ^ p)
        x >>= 4
    return p

mac_x1 = format(ceasar_cbc_mac(0b1101_1111_1010_0001), "#06b")
print(f"CBC-MAC(x ) = {mac_x1}")
mac_x2 = format(ceasar_cbc_mac(0b0010_1100_0001_1111), "#06b")
print(f"CBC-MAC(x') = {mac_x2}")

CBC-MAC(x ) = 0b1111
CBC-MAC(x') = 0b0010


# Oppgave 3

In [199]:
from collections import deque
from itertools import islice
lfsr_keys = ((1,0,0,0), (0,0,1,1), (1,1,1,1))
lfsr_max_period = 2**4 - 1

def lfsr1(z: deque[int]) -> int:
    return sum(z) & 1

def lfsr2(z: deque[int]) -> int:
    return (z[0] + z[-1]) & 1

def lfsr_generator(key: tuple[int, ...], lfsr_f: Callable[[deque[int]], int]):
    q = deque((key))
    while True:
        q.append(lfsr_f(q))
        yield q.popleft()
        
for lfsr_n, lfsr_f in enumerate((lfsr1, lfsr2)):
    print(f"LFSR #{lfsr_n+1}:")
    for i, key in enumerate(lfsr_keys):
        nums = "".join(map(lambda x: str(x), islice(lfsr_generator(key, lfsr_f), lfsr_max_period)))
        print(f"    Nøkkel #{i+1} => {nums}...")

LFSR #1:
    Nøkkel #1 => 100011000110001...
    Nøkkel #2 => 001100011000110...
    Nøkkel #3 => 111101111011110...
LFSR #2:
    Nøkkel #1 => 100011110101100...
    Nøkkel #2 => 001111010110010...
    Nøkkel #3 => 111101011001000...


## a)
Fra kode og resultatene ovenfor (LFSR #1) ser vi at periodene for alle nøklene har en periode på 5.

## b)
Fra kode og resultatene ovenfor (LFSR #2) ser vi at periodene for alle nøklene har maks-perioden lik 15.

# Oppgave 4
## a)

In [200]:
rsa_p = 32771
rsa_q = 65519
rsa_e = 3
rsa_n = rsa_p*rsa_q

rsa_pub = (rsa_n, rsa_e)

# Source: https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/
def mult_inverse(a: int, b: int) -> int:
    if a == 0:
        return b, 0, 1
    
    gcd, x1, y1 = gcd_extended(b % a, a)
    x = y1 - (b//a) * x1
    y = x1
    if x < 0:
        x += b
    return x

def exponentiate(a: int, e: int, m: int) -> int:
    if e == 0:
        return 1
    x = exponentiate((a*a) % m, e // 2, m) % m
    if e & 1 == 1:
        x *= a
    return x % m


## b)
Jeg har valgt to 16-bits primtall $p=32771$ og $q=65519$, slik at $(p-1)(q-1)$ ikke hadde $3$ som en faktor. Dette betyr at $e$ er relativt primt til $(p-1)(q-1)$.

## c)

In [201]:
print(rsa_pub)

(2147123149, 3)


## d)

In [202]:
rsa_d = mult_inverse(rsa_e, (rsa_p-1)*(rsa_q-1))
rsa_prv = (rsa_p, rsa_q, rsa_d)
print(rsa_prv)

(32771, 65519, 1431349907)


## e)

In [203]:
rsa_msg = 12345
rsa_encrypted = exponentiate(rsa_msg, rsa_e, rsa_n)
print(f"Kryptert: {rsa_encrypted}")
rsa_decrypted = exponentiate(rsa_encrypted, rsa_d, rsa_n)
print(f"Dekryptert: {rsa_decrypted}")

Kryptert: 486085101
Dekryptert: 12345


# Oppgave 5

In [204]:
from math import gcd

def pollard(n, b):
    a = 2
    for i in range(2, b):
        a = exponentiate(a, i, n)
    d = gcd(a-1, n)
    if 1 < d < n:
        return d
    return None

## a)

In [205]:
pollard_f = pollard(1829, 5)
print(f"Primtallsfaktor i 1829: {pollard_f}")

Primtallsfaktor i 1829: None


# Oppgave 6
## a)
$$e_k(x)=x^b \mod{n}$$
$$e_k(x_1)e_k(x_2)=x_1^b x_2^b \mod{n} = (x_1 x_2)^b \mod {n}$$
$$Potensregel: x_1^b x_2^b = (x_1 x_2)^b$$
$$\implies e_k(x_1)e_k(x_2) \mod{n} = e_k(x_1 x_2) \mod{n}$$

## b)
Eva kjenner til n fra offentlig nøkkel.
$$ y' = y*r^e $$
$$ x' = d_k(y') = d_k(y \cdot r^e) = (y \cdot r^e)^d = (e_k(x) \cdot r^e)^d = (x^e \cdot r^e)^d = x^{ed} \cdot r^{ed} \mod n = x \cdot r \mod n$$
Eva kan nå bruke $r^{-1}$ for å finne x
$$ x \cdot r \cdot r^{-1} \mod n = x \mod n$$

$$ $$